In [1]:
import sys
sys.path.append('../../')
sys.path.append('../../../flaming-choripan')

In [8]:
%load_ext autoreload
%autoreload 2
from flamingchoripan.files import load_pickle, get_filedirs
from flamingchoripan.datascience.statistics import XError
import numpy as np

class Rank():
    def __init__(self, name,
        print_n=10):
        self.name = name
        self.print_n = print_n
        self.names = []
        self.values = []
        
    def add(self, name, value):
        self.names.append(name)
        self.values.append(value)
        
    def calcule_rank(self):
        self.idxs = np.argsort(self.values)[::-1].tolist()
        
    def __len__(self):
        return len(self.names)

    def __repr__(self):
        txt = f'{self.name}:\n'
        idxs = self.idxs
        for k,idx in enumerate(idxs):
            txt += f'({k+1}) - {self.names[idx]}: {self.values[idx]}\n'
            if k+1>=self.print_n:
                break
        return txt[:-1]
    
methods = ['curve_fit']
band_names = ['g', 'r']
rootdir = '/home/opimentel/tesis/astro-lightcurves-handler/save/alerceZTFv5.1'

results_dict = {}
rank = {}
rank_bdict = {}
for method in methods:
    results_dict[method] = {
        'mb-fit-error':[],
        'mb-null-fits':0,
    }
    for b in band_names:
        results_dict[method].update({
            f'{b}-fit-error':[],
            f'{b}-null-fits':0,
        })

    rank[method] = Rank('mb-rank')
    rank_bdict[method] = {b:Rank(f'{b}-rank') for b in band_names}

### loop
for method in methods:
    load_rootdir = f'{rootdir}/{method}'
    filedirs = get_filedirs(load_rootdir, fext='ferror')
    ov_bdict = {b:0 for b in band_names}
    for filedir in filedirs:
        fdict = load_pickle(filedir, verbose=0)
        lcobj_name = fdict['lcobj_name']
        xes = []
        for b in band_names:
            ov = fdict['error_bdict'][b]
            ov_bdict[b] += len(ov)
            #print(len(ov), ov)
            v = [x for x in ov if x>0]
            if len(v)>0:
                xe = XError(v, 0)
                rank_bdict[method][b].add(lcobj_name, xe.mean)
                results_dict[method][f'{b}-fit-error'] += v
                results_dict[method][f'{b}-null-fits'] += len(v)
                results_dict[method][f'mb-fit-error'] += v
                results_dict[method][f'mb-null-fits'] += len(v)
                xes.append(xe)
                
        if len(xes)>0:
            rank[method].add(lcobj_name, np.mean([xe.mean for xe in xes]))
            
for method in methods:
    results_dict[method]['mb-fit-error'] = XError(results_dict[method]['mb-fit-error'], 0)
    total_ov = sum([ov_bdict[b] for b in band_names])
    results_dict[method]['mb-null-fits'] = (1-results_dict[method]['mb-null-fits']/total_ov)*100
    for b in band_names:
        results_dict[method][f'{b}-fit-error'] = XError(results_dict[method][f'{b}-fit-error'], 0)
        results_dict[method][f'{b}-null-fits'] = (1-results_dict[method][f'{b}-null-fits']/ov_bdict[b])*100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
for method in methods:
    print(results_dict[method])

{'mb-fit-error': XError(2.54+-15.39, dim=0), 'mb-null-fits': 11.931085149082566, 'g-fit-error': XError(2.26+-11.86, dim=0), 'g-null-fits': 18.42567373853211, 'r-fit-error': XError(2.78+-17.88, dim=0), 'r-null-fits': 5.436496559633031}


In [13]:
for method in methods:
    print(method)
    rank[method].calcule_rank()
    print(rank[method])
    for b in band_names:
        rank_bdict[method][b].calcule_rank()
        print(rank_bdict[method][b])

curve_fit
mb-rank:
(1) - ZTF18acvvppd: 305.3215026855469
(2) - ZTF18aboebre: 244.23275756835938
(3) - ZTF19aajxwnz: 71.73245239257812
(4) - ZTF19aacgslb: 52.663902282714844
(5) - ZTF18achaqmd: 43.40021514892578
(6) - ZTF19aamsetj: 35.71979522705078
(7) - ZTF18abtfgqr: 31.628253936767578
(8) - ZTF19aampllw: 22.369037628173828
(9) - ZTF18abltdkd: 20.69577980041504
(10) - ZTF18acrnfww: 19.955039978027344
g-rank:
(1) - ZTF18acvvppd: 233.5023651123047
(2) - ZTF18aboebre: 148.15115356445312
(3) - ZTF19aajxwnz: 71.73245239257812
(4) - ZTF19aamsetj: 51.98676681518555
(5) - ZTF18achaqmd: 43.40021514892578
(6) - ZTF18acrnfww: 38.3190803527832
(7) - ZTF18acslpba: 28.983369827270508
(8) - ZTF19aatlmbo: 28.553916931152344
(9) - ZTF19aacgslb: 22.98574447631836
(10) - ZTF19abdzehg: 22.46877670288086
r-rank:
(1) - ZTF18acvvppd: 377.1406555175781
(2) - ZTF18aboebre: 340.3143615722656
(3) - ZTF19aacgslb: 82.34205627441406
(4) - ZTF19aampllw: 36.19752502441406
(5) - ZTF18abtfgqr: 31.628253936767578
(6) -